In [17]:
from typing import Tuple
import requests
from config import Config
import json


def get_bars_in_range(location: Tuple[float, float], radius: int, time: str = None):
    """
    location: Tuple[float, float]
    radius: int
        Keresés rádiusza m-ben megadva. 
        'Doing so instructs the Places service to prefer showing results within that circle; results outside of the defined area may still be displayed.'
    time:str
    """
    conf = Config()

    next_page_token = None
    paging_token_hash = {}
    places = list()
    for i in range(25):

        if next_page_token is None:
            url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location[0]}%2C{location[1]}&radius={radius}&type=bar&key={conf.PLACES_API_KEY}"
        else:
            url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location[0]}%2C{location[1]}&radius={radius}&type=bar&page_token={next_page_token}&key={conf.PLACES_API_KEY}"

        payload = {}
        headers = {}

        # TODO handle errors
        response = requests.request("GET", url, headers=headers, data=payload)

        #this part should be rewritten with clear head
        res = json.loads(response.text)
        places += res['results']
        if 'next_page_token' in res:
            next_page_token = res['next_page_token']
            if next_page_token in paging_token_hash:
                break
            paging_token_hash[next_page_token] = None
        else:
            next_page_token = None
        if next_page_token is None:
            break
    return places


In [18]:
I_location = (47.47254804521419, 19.059588492595783)
bars = get_bars_in_range(I_location,1000)


AW30NDwWEvpp0wtFgaKNiiwwtJJZpG08AKxkNLu2zlbTzuxh1McSPdSKH6XQbIqgv0FeUXz1IKfgcK9xmXkNmVxt8Y9smpOe85noI5Aa5gD-f3MeILNHn8jwjofGzmDdh8SA1mnzx_4K2uTYQRik1NhAxDTzMs5-8299vVKoscia7inGc9YNEMH1-QJ2OIG9rAXmLEH0cKRDPIB6h8gXW0b9x03cn5n69e4NvDzcqjnVFBwk4d8YwTfLAl-lMJ5Hy9FffPnf7Jo1Cjj5_cGp9S3xnDCsl5WGKHClg-6k4AQ1TrRmQusvmLzIcNUgTJSOyzXrMgQ7hNYBbEK_LK40MqZhlR0r_yu6i97otemdOAV5PFDAeLFCa75UWRTQoZPAjc5zdehlzTWf3MXQ7a5LDMMOX_WCPIbmYwfhS5kd3ensW3zg0W4xOdI
AW30NDxqnAwDMiyxfOaFPp9Jz4zpmNI52BTsjVbB2xbamJl4GiQe2-tGMJOOks1cFN1rpTfG__IzlJxM9IWczSUoAUZBZ7m-3W3YWtk6KVrcCVAnJdLHmdJSwXyOKXyjQn-89wZo-CIhSDf7sahNPNF0ON3RP0zDJSVlsg8crYlz9FeX6IMc87p9134zCCKrqyMMwIkBnd_EAMfkcVswkFVGs6Ct_dLkmquBtNxTTOC1jDF8je0e_TSGPjjeYJZaH8Tr_AkDQnv5PR3GFMhP6KPh0917wtWtqX3rx6VzjL2itLYmrXwpCwlVQZi-sP90RHRZ5-KVKUDYfgNOmBhI-JX5owSHSynPUkVxU4enY0yNlZ78slqsoSjYn9gx9Hhk2r6gBQhl5vDfObLSkPoLU4zJQl0jq3VFwGKLe7Pzt3ZctWThzoSCd4Y
AW30NDwbc-V7hhWrKtGAMksL1rmJjzl-U_ZK-ZLRl4BaP7jWFvfWXbMeXw5EuEOb9-iN-SYYya-sRJDcfaCRJ4DEICh6GRNyXhhEIY_kTlDk1RMJuikZXrl5

In [8]:
keys = [k for k in bars[0]]
print(keys,len(bars))

['business_status', 'geometry', 'icon', 'icon_background_color', 'icon_mask_base_uri', 'name', 'opening_hours', 'photos', 'place_id', 'plus_code', 'price_level', 'rating', 'reference', 'scope', 'types', 'user_ratings_total', 'vicinity'] 500


In [9]:
for bar in bars[0:5]:
    print(f"{bar['name']} id={bar['geometry']}")

Napközi Bisztró id={'location': {'lat': 47.4731179, 'lng': 19.0517008}, 'viewport': {'northeast': {'lat': 47.4744993802915, 'lng': 19.0531154302915}, 'southwest': {'lat': 47.4718014197085, 'lng': 19.0504174697085}}}
Pinyó Klub id={'location': {'lat': 47.4766196, 'lng': 19.0527181}, 'viewport': {'northeast': {'lat': 47.4778401802915, 'lng': 19.0539852802915}, 'southwest': {'lat': 47.4751422197085, 'lng': 19.0512873197085}}}
Black Dog Pub id={'location': {'lat': 47.4778761, 'lng': 19.0517862}, 'viewport': {'northeast': {'lat': 47.47918508029149, 'lng': 19.0531151302915}, 'southwest': {'lat': 47.47648711970849, 'lng': 19.0504171697085}}}
Fruitisimo id={'location': {'lat': 47.4750225, 'lng': 19.0489757}, 'viewport': {'northeast': {'lat': 47.4764043802915, 'lng': 19.0507104802915}, 'southwest': {'lat': 47.4737064197085, 'lng': 19.0480125197085}}}
Happy Drink Bar id={'location': {'lat': 47.4775751, 'lng': 19.0511132}, 'viewport': {'northeast': {'lat': 47.4790883802915, 'lng': 19.052544780291

In [20]:
import haversine as hs
bar_hash = dict()
for bar in bars:
    loc = (bar['geometry']['location']['lat'],bar['geometry']['location']['lng'])
    if bar['name'] not in bar_hash:
        bar_hash[bar['name']] = 1
    else:
        bar_hash[bar['name']] += 1
    print(f"{bar['name']} dist: {hs.haversine(I_location,loc,unit=hs.Unit.METERS):4.5}")

print(bar_hash)

Napközi Bisztró dist: 596.22
Pinyó Klub dist: 686.74
Black Dog Pub dist: 833.59
Fruitisimo dist: 843.78
Happy Drink Bar dist: 847.48
Oportó Borozó dist: 854.16
Műhely Pub dist: 863.96
Mag Pub dist: 906.9
Clock Brasserie dist: 897.17
Bercsényi Borozó Bt. dist: 906.9
Sarki Tanszék dist: 930.66
Fapados Söröző Borozó dist: 978.7
Szatyor Bár dist: 984.67
Kamra Campus dist: 372.38
Eufória terasz dist: 102.17
Szertár dist: 167.86
Spaten dist: 465.69
Karman Club dist: 476.18
Eleven Bistro Budapest dist: 677.49
Duna Bár Kft. dist: 683.14
Napközi Bisztró dist: 596.22
Pinyó Klub dist: 686.74
Black Dog Pub dist: 833.59
Fruitisimo dist: 843.78
Happy Drink Bar dist: 847.48
Oportó Borozó dist: 854.16
Műhely Pub dist: 863.96
Mag Pub dist: 906.9
Clock Brasserie dist: 897.17
Bercsényi Borozó Bt. dist: 906.9
Sarki Tanszék dist: 930.66
Fapados Söröző Borozó dist: 978.7
Szatyor Bár dist: 984.67
Kamra Campus dist: 372.38
Eufória terasz dist: 102.17
Szertár dist: 167.86
Spaten dist: 465.69
Karman Club dist: 